In [2]:
DATADIR = r'E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset'
SIGNALS = ["body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x",
           "total_acc_y", "total_acc_z"]

In [4]:

import pandas as pd
import numpy as np


def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        print(filename)
        signals_data.append(_read_csv(filename).as_matrix()) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test


In [ ]:
load_signals('train')

In [6]:
f1=r'E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt'
df=_read_csv(f1)

In [ ]:
signals_data = []
subset='train'
for signal in SIGNALS:
    filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
    print(filename)
    signals_data.append(_read_csv(filename).as_matrix())

In [13]:
np.transpose(signals_data).shape

(128, 7352, 9)

In [ ]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])


In [ ]:

X_train, X_test, Y_train, Y_test = load_data()


In [14]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

# for reproducibility
# https://github.com/fchollet/keras/issues/2280
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout


epochs = 30
batch_size = 16
n_hidden = 32

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

X_train, X_test, Y_train, Y_test = load_data()

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

model.fit(X_train,Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

# Evaluate
print(confusion_matrix(Y_test, model.predict(X_test)))

F:\environment\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt
E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt
E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt
E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt
E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt
E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/body_gyro_z_train.txt
E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt
E:\my_proj\fog_recognition\fog_optimization\dl4fog\resources\UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt
E:\my_proj\fog_recognition\

NameError: name 'confusion_matrix' is not defined